In [5]:
import torch
import numpy as np
from torch.utils import data
from d2l import torch as d2l
import random

# 生成数据集
def synthetic_data(w, b, num_examples):
    # 生成1000个样本数据：shape为1000 * 2样本矩阵
    X = torch.normal(0, 1, (num_examples, len(w)))
    # print(X 
    y = torch.matmul(X, w) + b
    # print(y)
    y += torch.normal(0, 0.01, y.shape)
    # y =Xw + b + c c为潜在观测误差
    return X, y.reshape(-1, 1)


true_w = torch.tensor([2, -3.4])
true_b = 4.2
features, labels = synthetic_data(true_w, true_b, 1000)
print(features)  # 样本数据
print(labels)  # 标签

tensor([[-0.6944,  0.7738],
        [ 1.6396, -0.2964],
        [ 0.4554,  0.4080],
        ...,
        [-0.0500,  1.5169],
        [ 0.6530, -0.4187],
        [ 0.5703, -0.6491]])
tensor([[ 0.1954],
        [ 8.4653],
        [ 3.7089],
        [ 3.9456],
        [-0.6695],
        [13.9770],
        [-0.3597],
        [-2.8175],
        [-1.4223],
        [ 8.8412],
        [ 6.7896],
        [ 6.7487],
        [ 5.3292],
        [ 4.5782],
        [10.6526],
        [ 1.4518],
        [ 6.7212],
        [ 1.5963],
        [ 4.0646],
        [ 4.6775],
        [10.3512],
        [ 6.9679],
        [ 5.2602],
        [ 2.0682],
        [-0.1208],
        [ 2.7019],
        [10.8969],
        [ 5.0784],
        [ 4.9086],
        [ 9.8659],
        [ 6.7782],
        [ 8.2578],
        [ 1.6140],
        [-2.0205],
        [ 0.0237],
        [13.2781],
        [ 4.2559],
        [10.1497],
        [ 5.5568],
        [-0.6380],
        [ 9.9893],
        [-2.7559],
        [ 9.2844],
 

In [9]:
# 小批量抽取样本数据
def data_iter(batch_size, features, labels):
    num_examples = len(features)
    indices = list(range(num_examples)) 
    # 这些样本是随机读取的，没有特定的顺序
    random.shuffle(indices)  # 打乱索引顺序
    for i in range(0, num_examples, batch_size):
        batch_indices = torch.tensor(indices[i : min(i + batch_size, num_examples)])
        yield features[batch_indices], labels[batch_indices]

batch_size = 10

for X, y in data_iter(batch_size, features, labels):
    print(X, "\n", y)
    break

tensor([[ 0.4694, -0.3411],
        [ 0.1128, -1.4113],
        [ 1.3694, -0.1044],
        [ 0.5780,  0.9316],
        [ 0.0655, -1.1766],
        [-1.6838, -0.4678],
        [-2.5279,  0.5063],
        [ 0.2774, -1.3928],
        [ 1.2147,  0.3311],
        [-1.6394,  0.0222]]) 
 tensor([[ 6.2946],
        [ 9.2224],
        [ 7.2859],
        [ 2.1897],
        [ 8.3244],
        [ 2.4374],
        [-2.5702],
        [ 9.4715],
        [ 5.5048],
        [ 0.8315]])


In [ ]:
# 初始化模型参数
w = torch.normal(0,0.01,size=(2,1),requires_grad=True)
b = torch.zeros(1, requires_grad=True)
print(w)
print(b)


tensor([[ 0.0202],
        [-0.0137]], requires_grad=True)
tensor([0.], requires_grad=True)


In [ ]:
# 定义模型
def linreg(X, w, b):
    """线性回归模型"""
    return torch.matmul(X, w) + b

# 定义损失函数
def square_loss(y_hat, y):
    """均方损失"""
    return (y_hat - y.reshape(y_hat.shape)) ** 2 / 2

# 定义优化算法
def sgd(params, lr, batch_size):
    """小批量随机梯度下降"""
    with torch.no_grad():  # 用来临时关闭 PyTorch 的自动求导机制
        for param in params:
            param -= lr * param.grad / batch_size
            param.grad.zero_()

lr = 0.03  
num_epochs = 3
net = linreg
loss = square_loss

for epoch in range(num_epochs):
    for X, y in data_iter(batch_size, features, labels):
        l = loss(net(X, w, b), y)  # X和y的小批量损失
        # 因为l形状是(batch_size,1)，而不是一个标量。l中的所有元素被加到一起，
        # 并以此计算关于[w,b]的梯度
        l.sum().backward()
        sgd([w, b], lr, batch_size)  # 使用参数的梯度更新参数

    # 输出查看w,b更新后的损失值
    with torch.no_grad():
        train_l = loss(net(features, w, b), labels)
        print(f'epoch {epoch + 1}, loss {float(train_l.mean()):f}')

epoch 1, loss 0.042961
epoch 2, loss 0.000167
epoch 3, loss 0.000046
